<a href="https://colab.research.google.com/github/rcanzlovar/Jupyterdocs/blob/master/Facebook_Summarizer_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facebook Summarizer

Facebook has a facility to allow you to download all of your facebook information. 

* Go to Facebook.com/settings
* Tap “Download a copy of your Facebook data.”
* Tap “Download Archive.”

    It might take a few minutes, but Facebook will alert you when your archive is ready.
* When it is, click “Download Archive” again, and a zip file will download to your computer. The download will be a big zip file that has all your posts, all your pictures, and all kinds of other stuff. 

In the directory structure that is created, ./posts/your_posts.json contains all of your posts: updating statuses, sharing links, photos, etc. along with comments to these. 

If you are just reposting a link from someone else, the link will not be carried, but your comment is. 

In breakoutdate() this script parses out your_posts.json and breaks out the month and day of each post im Markdown format so that the next step (daybook) can create a series of up to 366 posts for each day of the year so that all january 1 posts are together, then all January 2, etc. Currently we create bogus ones like 04_31 and 02_31 but they should be empty.


In [13]:
import pandas as pd

# load google drive to save things... 
from google.colab import drive
drive.mount('/content/drive')

file = "/content/drive/My Drive/fbfun/soltahr_posts.json"
#file = "/content/drive/My Drive/fbfun/rca_posts.json"

file_prefix = "/content/drive/My Drive/fbfun/soltahr/soltahr_post_"


## use this to upload a file to known file name. It's a little clunky 
#from google.colab import files
#uploaded = files.upload()

#print ("uploaded",uploaded)
import io
df = pd.read_json(file)
# Dataset is now stored in a Pandas Dataframe

import datetime 
# mangle the timestamps to be date objects 
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#rm  /content/drive/My\ Drive/fbfun/soltahr/*.md
#!bash


In [15]:
# make transformations:  running average 
days = ["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]
import math

def breakoutdate (mydf):
  for i in range(0,mydf.shape[0]):
    url = title = post = ''
    timestamp = mydf.iloc[i].timestamp
    ts = pd.to_datetime(timestamp, errors='coerce')
    try:
      post =  mydf.iloc[i].data[0]['post']
    except TypeError:
      post = '???'
    except KeyError:
      post = '???!'
    mydf.loc[mydf.index[i],'post'] = post
    #
    try:
      title = mydf.iloc[i].title
    except:
      title = post
   
    # URL attachment
    # note this can be a duplicate of a URL that is part of the post
    # it seems that once facebook recognizes it as a URL then it makes
    # the URL element

    # need to do a curl or wget to get
    try:
        url = mydf.iloc[i]['attachments'][0]['data'][0]['external_context']['url']
        if url:
          mydf.loc[mydf.index[i],'url'] = url
    except:
      pass

    comments = []
    # picture uploads
    try:
       media_uri = mydf.iloc[i]['attachments'][0]['data'][0]['media']['uri']
       mydf.loc[mydf.index[i],'media'] = media_uri
    except:
      pass

    mydf.loc[mydf.index[i],'weekday'] = ts.weekday()
    mydf.loc[mydf.index[i],'month'] = ts.month
    mydf.loc[mydf.index[i],'day'] = ts.day

def foo():
    print(days[mydate.weekday()])
    

    # iterate over comments
    for x in range(len(comments)): 
        tx = datetime.datetime.fromtimestamp(comments[x]['timestamp'])
        print("* ",tx,comments[x]['author'])
        print(comments[x]['comment'])
        print("---")



    print("---")



In [16]:
breakoutdate(df)

In [17]:


import pprint as pp
import sys

def print_to_stderr(*a): 
    # Here a is the array holding the objects 
    # passed as the arguement of the function 
    print(*a, file = sys.stderr) 
  

!pip install pypandoc
import pypandoc 

def daybook(df,month,day):

    df2 = df[df['day'] == day]
    df3 = df2[df2['month'] == month]
    datestring = ('','0')[month < 10] + str(month) + "_" + ('','0')[day < 10] +  str(day)

    filebase = file_prefix + datestring
    myinfile = filebase + '.md'
    myoutfile = filebase + '.html'

    print_to_stderr ("out to file ",myinfile)
    print ("out to file ",myinfile)
    original_stdout = sys.stdout # Save a reference to the original standard output

###    if 1==1: # useful for troublshooting when i've moved stdout to a file 
    with open(myinfile, 'w') as f:
        sys.stdout = f # Change the standard output to the file we created.
        #   sys.stdout = f # Change the standard output to the file we created.

        ####with open(file, 'w') as f:
        ####   sys.stdout = f # Change the standard output to the file we created.
        #  with open(file, 'w') as f:
        #    sys.stdout = f # Change the standard output to the file we created.
        print ('<a name="' + datestring + '"></a>')

        print ('<link rel="stylesheet" type="text/css" href="style.css">')


        # reverse for loop because facebook stores this newst to oldest
        for i in range(df3.shape[0] - 1, -1, -1):
#        for i in range(0,df3.shape[0]):
            #print_to_stderr(df3.iloc[i])
            thing = df3.iloc[i]

            #title
            if thing['title'] == thing['title']:
              title = thing['title']
            else:
              title = ""

            # if someone posts a photo to your timeline, you dont get anything. skip it 
            if (title.find('photo to your timeline') > 0):
              continue
            #if (thing['post'].find('???') > 0):
            #  continue
            if (title.find('photo to your timeline') > 0):
              continue
            if (title.find('shared a') > 0):
              if (thing['post'].find('???') > 0 and  
                  thing['url'] != thing['url'] and 
                  thing['media'] != thing['media']):
                
                      print_to_stderr("post = ",thing['post'])
                      print_to_stderr("url = ",thing['url'])
                      print_to_stderr("media = ",thing['media'])
                      print_to_stderr("=====")

                      print_to_stderr("SKIPPING")
                      print_to_stderr(thing)


                      continue

            # 
            # hr separator 
            print('<div class="content" width="400px">')

            print('---')
            print()

            print ("### " + str(thing['timestamp']) + " - " + title)
            print()
            if thing['post']:
              if thing['post'] != '???':
                print (thing['post']) 

            if thing['url'] == thing['url']:
              print ('[' + thing['url'] + '](' + thing['url'] + ')')  
            if thing['media'] == thing['media']:
              #print ('![image](' + thing['media'] +')')  
              print ('<img src="' + thing['media'] + '" width="100%">')  
            print('</div>')


        sys.stdout = original_stdout # Reset the standard output to its original value
        #### sys.stdout = original_stdout # Reset the standard output to its original value
        # sys.stdout = original_stdout # Reset the standard output to its original value
###        f.close() # is this necessary?
        #pypandoc.convert_file(myinfile, 'html',outputfile=myoutfile)



In [18]:
#!rm "/content/drive/My Drive/fbfun/soltahr/*"
#!bash


In [19]:

for mo in range(1,12+1):
  for da in  range(1,31+1):
    daybook(df,mo,da)

#daybook(df,1,1)
print_to_stderr("and on to the next thing")

out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_01.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_02.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_03.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_04.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_05.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_06.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_07.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_08.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_09.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_10.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_11.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_12.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_13.md


out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_01.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_02.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_03.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_04.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_05.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_06.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_07.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_08.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_09.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_10.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_11.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_12.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_13.md


out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_14.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_15.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_16.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_17.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_18.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_19.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_20.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_21.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_22.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_23.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_24.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_25.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_26.md
out to file  /content/drive/My Drive/fbfun/soltahr/

out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_14.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_15.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_16.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_17.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_18.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_19.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_20.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_21.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_22.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_23.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_24.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_25.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_26.md
out to file  /content/drive/My Drive/fbfun/soltahr/

out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_28.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_29.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_30.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_01_31.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_01.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_02.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_03.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_04.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_05.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_06.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_07.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_08.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_02_09.md


...

out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_20.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_21.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_22.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_23.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_24.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_25.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_26.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_27.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_28.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_29.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_30.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_31.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_01.md
out to file  /content/drive/My Drive/fbfun/soltahr/

out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_20.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_21.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_22.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_23.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_24.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_25.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_26.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_27.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_28.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_29.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_30.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_11_31.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_01.md
out to file  /content/drive/My Drive/fbfun/soltahr/

out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_04.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_05.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_06.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_07.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_08.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_09.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_10.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_11.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_12.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_13.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_14.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_15.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_16.md
out to file  /content/drive/My Drive/fbfun/soltahr/

out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_04.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_05.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_06.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_07.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_08.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_09.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_10.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_11.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_12.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_13.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_14.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_15.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_16.md
out to file  /content/drive/My Drive/fbfun/soltahr/

out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_19.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_20.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_21.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_22.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_23.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_24.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_25.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_26.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_27.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_28.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_29.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_30.md
out to file  /content/drive/My Drive/fbfun/soltahr/soltahr_post_12_31.md
and on to the next thing


In [20]:
#!bash


In [21]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.huffpost.com/entry/girltrek-harriet-tubman-walking-journey_n_5a9ff20ae4b0e9381c1433b5'

response = requests.get(url)
soup = BeautifulSoup(response.text)

metas = soup.find_all('meta')

print ([ meta.attrs['content'] for meta in metas if 'name' in meta.attrs and meta.attrs['name'] == 'description' ])

[]
